This is the notebook of Milestone 2 of team _Ada And the Lovelaces_. The outline of the notebook is as followed:
1. Data Pre-Processing: we add the features of interest to the existing datasets
2. Exploratory Data Analysis: based on the datasets obtained, we run some first basic analysis and visualizations


In [16]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import urllib.parse as urlparse
from collections import defaultdict

from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re
import os
import networkx as nx

# Data pre-processing

## Computing clicks, impressions and CTR

This is done at two different granularities:
- Per source: we compute the clicks, impressions and CTR for each outgoing links of a given article
- Aggregated: we compute the total number of clicks, impressions and CTR for the articles no matter the origin of the user's click

### Fine grained dataset: per source article
Here, the goal is to extend the _links_ table by adding for each pair _(origin article, destination article)_, columns counting the number of clicks associated, the number of impressions, and the CTR.

In [17]:
# Helper functions

def title_parse(title):
    """
    Parse URL encoded to readable characters
    """

    title = urlparse.unquote(title).casefold()
    return title



def url_encode(title):
    """
    URL encode target links
    """

    title = urlparse.quote(title)
    return title


We start by retrieving the players's data: they are in the paths_finished and paths_unfinished datasets. We first focus on the finished paths:

In [18]:
df_user_finished = pd.read_csv('./data/wikispeedia_paths-and-graph/paths_finished.tsv',
                      sep='\t',
                      names=['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating'],
                      comment='#')

# It was noticed that users clicked on links in the footer (Terms & Conditions + Disclaimer) 
# that are not articles per-say. We decided to remove all paths containing these links as it is not in the spirit of the game
## "wikipedia_text_of_the_gnu_free_documentation_license" is not an article!
problematic_keywords = ['Wikipedia_Text_of_the_GNU_Free_Documentation_License']
df_user_finished = df_user_finished[~df_user_finished.path.str.contains('|'.join(problematic_keywords))]

# Drop unused columns for objectives and transform to list of paths
df_user_finished = df_user_finished.drop(['hashedIpAddress', 'timestamp', 'durationInSec', 'rating'], axis=1).squeeze()


We now enrich our series_paths dataframe with the unfinished paths

In [19]:
df_user_unfinished = pd.read_csv('./data/wikispeedia_paths-and-graph/paths_unfinished.tsv',
                      sep='\t',
                      names=["hashedIpAddress", "timestamp", "durationInSec", "path", "target", "type"],
                      comment='#')

# It was noticed that users clicked on links in the footer (Terms & Conditions + Disclaimer) 
# that are not articles per-say. We decided to remove all paths containing these links as it is not in the spirit of the game
## "wikipedia_text_of_the_gnu_free_documentation_license" is not an article!
problematic_keywords = ['Wikipedia_Text_of_the_GNU_Free_Documentation_License']

df_user_unfinished = df_user_unfinished[~df_user_unfinished.path.str.contains('|'.join(problematic_keywords))]

df_user_unfinished = df_user_unfinished.drop(['hashedIpAddress', 'timestamp', 'durationInSec', 'target', 'type'], axis=1).squeeze()

In [20]:
df_user = df_user_finished.append(df_user_unfinished)#Need to chance to pandas concat TODO
# TODO Remove problematic artilces here ONCE
# Decode URL encoding for paths, case fold
series_paths = df_user.map(title_parse)

# Transform paths to list of inputs (article names or <)
series_paths = series_paths.str.split(';')

C:\Users\Hugo Parent\AppData\Local\Temp\ipykernel_8076\32903713.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_user = df_user_finished.append(df_user_unfinished)#Need to chance to pandas concat TODO


We add the new columns of interest in the df_links dataframe, and initialize them:

In [29]:
# Extract list of links as DataFrame to store frequency

df_links = pd.read_csv('./data/wikispeedia_paths-and-graph/links.tsv',
                       sep='\t',
                       names=['linkSource_URL', 'linkTarget_URL'],
                       comment='#')
# Definitions
# Source Links are the link of the page on which a link is shown
# Target Links are the outgoing link on which the user lands

# Filenames are single encoded (ie C++.htm is encoded at C%2B%2B.htm)
# Outgoing names in the HTML source are DOUBLE encoded (ie C++.htm is <a href="../../wp/c/C%252B%252B.htm">
# Therefore, we add a helper column for linkTarget_2URL storing the DOUBLE URL ENCODED string
df_links['linkTarget_2URL'] = df_links['linkTarget_URL'].map(url_encode)
df_links['linkSource_2URL'] = df_links['linkSource_URL'].map(url_encode)

# Decode URL encoding for paths, case fold (from URL encoding to readable)
df_links['linkSource'] = df_links['linkSource_URL'].map(title_parse)
df_links['linkTarget'] = df_links['linkTarget_URL'].map(title_parse)

# Merge columns in tuple for easy lookup (unique key), format source;target
df_links['linkPair'] = df_links['linkSource'] + ';' + df_links['linkTarget']

# Number of impressions will be counted in column 'impressions' for each pair
df_links['impressions'] = pd.Series(np.zeros((len(df_links)),dtype=int))

# Number of hits will be counted in column 'hits' for each pair
df_links['clicks'] = pd.Series(np.zeros((len(df_links)),dtype=int))

# Storing the x and y position of the unique link on the page
df_links['xpos'] =  pd.Series(np.zeros((len(df_links)),dtype=int))
df_links['ypos'] =  pd.Series(np.zeros((len(df_links)),dtype=int))

In [30]:
# Extract list of articles
df_articles = pd.read_csv('./data/wikispeedia_paths-and-graph/articles.tsv',
                          sep='\t',
                          names=['title'],
                          comment='#')
series_articles_URL = df_articles['title']
series_articles_2URL = series_articles_URL.map(url_encode)
# Decode URL encoding for paths, case fold
series_articles = df_articles['title'].map(title_parse)

The next block of code serves for optimization.

In [31]:
# Optimization step, preprocessing indices to use numpy arrays for faster access
# For all articles names, create a list of rows in the df_links that have this article as source
source_map = defaultdict(list)
for link_id, source in enumerate(df_links['linkSource']):
    source_map[source].append(link_id)

# Create a reverse dictionary to index nparray
# list[int]=string to dict[string]=int
my_map = dict(df_links['linkPair'])
pair_map = dict((v, k) for k, v in my_map.items())

The computation of the number of clicks and impressions:

In [32]:
np_links = df_links.to_numpy()
IMPRESSIONS_COLUMN_ID = df_links.columns.get_loc('impressions')
CLICKS_COLUMN_ID = df_links.columns.get_loc('clicks')
XPOS_COLUMN_ID = df_links.columns.get_loc('xpos')
YPOS_COLUMN_ID = df_links.columns.get_loc('ypos')

# For each path, count forward link clicks
for path in series_paths:
    # Initialise stack with first link
    foo = list()
    foo.append(path[0])
    # Iterate over every step of user's path
    for element in path[1:]:
        if element != '<':
            # If next element in path is not a return character, store information and analyze
            ## Source node is the current top of stack
            source_node = foo[-1]

            ## Add next link in list to top of stack
            foo.append(element)

            ## New top of stack is target
            target_node = foo[-1]

            # Create key for pair identification
            search_value = source_node + ';' + target_node

            # Some issues were discovered with the dataset and encoding: some link pairs are missing
            ### ie 'finland;åland' is not in our dataset, BUT the link 'åland' out of finland EXISTS on the source article and was used by the user
            # It was not listed as an outgoing link in the dataset
            try:
                pair = pair_map[search_value]
            except Exception as e:
                # Catch when pair was not listed in dataset, don't add impression and disregard
                print('Invalid pair:',e) # For debug and understanding dataset
                break

            ## Count one impression for all pairs with source_node as source (source_node;*)
            source = source_map[source_node]
            np_links[source,IMPRESSIONS_COLUMN_ID] += 1

            ## Add one click-through for the pair (source_node;target_node)
            np_links[pair,CLICKS_COLUMN_ID] += 1

        else:
            # If return character is read, pop top of stack and don't store any info
            foo.pop()

Invalid pair: 'finland;åland'
Invalid pair: 'finland;åland'
Invalid pair: 'republic_of_ireland;éire'
Invalid pair: 'republic_of_ireland;éire'
Invalid pair: 'claude_monet;édouard_manet'
Invalid pair: 'republic_of_ireland;éire'
Invalid pair: 'claude_monet;édouard_manet'
Invalid pair: 'republic_of_ireland;éire'
Invalid pair: 'finland;åland'
Invalid pair: 'finland;åland'
Invalid pair: 'ireland;éire'
Invalid pair: 'republic_of_ireland;éire'
Invalid pair: 'ireland;éire'
Invalid pair: 'impressionism;édouard_manet'
Invalid pair: 'claude_monet;édouard_manet'
Invalid pair: 'francisco_goya;édouard_manet'
Invalid pair: 'republic_of_ireland;éire'
Invalid pair: 'ireland;éire'
Invalid pair: 'ireland;éire'
Invalid pair: 'republic_of_ireland;éire'
Invalid pair: 'claude_monet;édouard_manet'
Invalid pair: 'republic_of_ireland;éire'


Let's put back the values obtained in np_links in the df_links dataframe, since dataframes are easier to handle than series

In [33]:
columns = df_links.columns
to_dict_values = {}


for i in range(len(columns)):

    col = columns[i]
    to_dict_values[col] = [np_links[k][i] for k in range(len(np_links))]

df_links = pd.DataFrame(to_dict_values)


Now, we can finally compute the CTR

In [34]:
df_links['CTR'] = df_links.clicks / df_links.impressions

## Computing the location of the links in the HTML pages

In this part, we compute the location of every link in a given HTML page. The location is added to the _links_ table.

Let's start by extracting the links, and creating the new columns we wish to add, with 0 as the initialization value:

Now, the HTML driver is created. We need to define arbitrary parameters. We consider a full screen display at 1900*900, rendered in Google Chrome.




_Note that if we were to change the resolution, the data would change but the ordering of the links locations on the y axis would remain unchanged. The results of the analysis would not change._

In [35]:
op = Options()
op.add_argument('headless')
op.add_argument("window-size=1920,900") # Most common window size for desktop, full-screen minus 130 at top for browser control, minus 50 at bottom for windows taskbar. Since the game was played, on desktop, we choose this (ADD EXPLANATORY SCREENSHOT)
driver = webdriver.Chrome(options=op)

Now that the driver is configured, let's compute the locations of all links in the articles

In [37]:
article_counter = 0
total = len(series_articles_2URL)

for source in series_articles_2URL:
    #source=series_articles_2URL[0] #DEBUG
    print('Current Article:',source,article_counter,'/',total )
    article_counter+=1 #For progress tracking

    #Compute relative path to html
    source_path = '/data/wpcd/wp/' + source[0].lower() + '/' + source + '.htm'

    #Absolute path to html file
    user_cwd = os.getcwd().replace('\\','/')
    source_url = 'file://'+user_cwd+source_path
    #Open the browser
    driver.get(source_url)

    #List of target links we want to find positions for
    list_target_articles = df_links.loc[df_links['linkSource_2URL']==source]['linkTarget_2URL']

    for target in list_target_articles:
        #The pairs are in readable characters, target is double encoded format
        target_readable = title_parse(title_parse(target))
        source_readable = title_parse(title_parse(source))
        search_pair = source_readable + ';' + target_readable

        try:
            current_pair = pair_map[search_pair]
        except Exception as e:
            # Catch when pair was not listed in dataset
            print('Invalid pair:',e) # For debug and understanding dataset
            break

        #Find the href with /target.htm in all elements of page
        #Example with target = World War I, we are looking for /World_War_I.htm
        href=driver.find_elements(By.XPATH,  ".//a[contains(@href,'/" + target + ".htm')]")
        try:
            df_links.loc[current_pair, 'xpos']=href[0].location['x']
            df_links.loc[current_pair, 'ypos']=href[0].location['y']
            #DEBUG
            #if len(href):
            #Rajouter dans une colonne occurence
            #print('Target',target,'appeared',len(href),'times')
        except Exception as e:
            print('Not found:',target)

Current Article: %25C3%2581ed%25C3%25A1n_mac_Gabr%25C3%25A1in 0 / 4604
Current Article: %25C3%2585land 1 / 4604
Current Article: %25C3%2589douard_Manet 2 / 4604
Current Article: %25C3%2589ire 3 / 4604
Current Article: %25C3%2593engus_I_of_the_Picts 4 / 4604
Current Article: %25E2%2582%25AC2_commemorative_coins 5 / 4604


KeyboardInterrupt: 

Saving the obtained dataset

In [14]:
df_links.to_csv('./output/output_links.csv',encoding='utf-8-sig')

### Coarse-grained dataset: Aggregated view over all website
Here, the goal is to extend the _articles_ table by adding columns representing the total number of clicks, the number of impressions and the CTR. Except for the CTR, we can simply sum the obtained numbers in the finer granularity.

Let's extract the articles data by aggregating the df_links dataframe

In [15]:
articles = df_links[["linkTarget", "impressions", "clicks"]].groupby(by=["linkTarget"]).sum()
articles["CTR"] = articles["clicks"]/articles["impressions"]

Let's save the result

In [16]:
articles.to_csv('./output/output_articles.csv',encoding='utf-8-sig')

# Exploratory Data Analysis

## Understanding the Wikispeedia network

To have a better understanding of the structure of the links, we wish to create a graph of the network.

In [17]:
G = nx.DiGraph()
G = nx.from_pandas_edgelist(df_links, source='linkSource', target='linkTarget')

In [18]:
nx.draw_networkx(G, with_labels=False, node_size=5, node_color="#001f3f", width=0.1, edge_color="#AAAAAA")

AttributeError: module 'scipy.sparse' has no attribute 'coo_array'

Interestingly, we distinguish two strongly separate connected components. We can verify that easily:

In [ ]:
print("There are " + str(nx.number_connected_components(G)) + "  connected components in the graph.")

In [ ]:
print("Unconnected components: "+ str(list(nx.connected_components(G))[1]))

This is linked to pages for donations, these are not Wikipedia articles and will be removed.

## Data Distribution

Let's explore the dataset we have obtained.

In [ ]:
print('There exist', len(df_links), 'links in total.','\n')
print('The unique articles are:', df_links.linkSource.nunique())
df_links['linkSource'].value_counts()

In [ ]:
group_by_links = df_links.groupby(by="linkSource").count().reset_index()[:50]

pl = group_by_links.plot(kind='bar', figsize=[11,5], rot=0)
pl.set_title('Distribution of the number of links per article')
pl.set_ylabel('Number of links')
pl.set_xlabel('Article')

We can see that the number of links per article does not follow a uniform distribution. There are articles that have more than 160 links while others have less than 20.

In [ ]:
articles["clicks"].plot.hist(bins=30, range=[0,1300], title="Number of clicks")

Let's rank the top 10 most clicks articles:

In [ ]:
articles.sort_values(by=['clicks'], ascending=False).head(10)

Now focusing on impressions:

In [ ]:
articles["impressions"].plot.hist(bins=30, title="Number of impressions", range=[0,50000])

Let's rank the top 10 articles with highest impressions

In [ ]:
articles.sort_values(by=['impressions'], ascending=False).head(10)

The finaly analysis is made on CTR:

In [ ]:
articles["CTR"].plot.hist(bins=30, title="Distribution of CTR", range=[0,0.75])

With the top 10 CTR ranking:

In [ ]:
articles.sort_values(by=['CTR'], ascending=False).head(10)

This last ranking might not be very useful since some articles have very low impressions, meaning we don't have enough data. The obtained CTR seem very high.

In [ ]:
print("The average number of clicks is " + str(articles.clicks.mean()))
print("The average number of impressions is " + str(articles.impressions.mean()))
print("The average CTR is " + str(articles.CTR.mean()))

## Sanity Check

Let's now check how clean how dataset is, and if we have missing data.